<a href="https://colab.research.google.com/github/vishnubharadwaj1234/Test-Prep-Study-Tool/blob/main/TestPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install openai==0.28

In [ ]:
import openai
import gradio as gr
import gradio.components as components
import os

In [ ]:
openai.api_key = "ENTER YOUR OPENAI API KEY HERE"

messages = [
    {"role": "system", "content": "You are a friendly and helpful AI assistant."},
]

def print_questions(subject, format):
    if subject:
        question = "Create 10 " + format + " questions for a test on " + subject + ". Do not write anything except for the questions."
        messages.append({"role": "user", "content": question})
        chat = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )
        reply = chat.choices[0].message.content
        messages.append({"role": "assistant", "content": reply})
        return reply

def print_answers():
    return "Hello world!"

# Gradio Interface
interface = gr.Interface(
              fn=print_questions,
              inputs=["text", gr.Dropdown(
                 choices=["Multiple choice", "True/False", "Fill in the blank", "Short answer", "Free response"],
                 allow_custom_value=True,
                 value = "Multiple choice",
                 label="Format of quiz", show_label=True
                 )],
              outputs=gr.Textbox(lines=30),
              title="Test Question Generator")

In [ ]:
interface.launch(share=True, debug=True)